In [6]:
url = 'https://new.land.naver.com/api/complexes/single-markers/2.0?cortarNo=4128112300&zoom=15&priceType=RETAIL&markerId&markerType&selectedComplexNo&selectedComplexBuildingNo&fakeComplexMarker&realEstateType=APT%3AABYG%3AJGC&tradeType=&tag=%3A%3A%3A%3A%3A%3A%3A%3A&rentPriceMin=0&rentPriceMax=900000000&priceMin=0&priceMax=900000000&areaMin=0&areaMax=900000000&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&directions=&leftLon=126.8048932&rightLon=126.8598248&topLat=37.6519089&bottomLat=37.6229543'

In [7]:
import requests
response = requests.get(url)

In [8]:
response

<Response [200]>

In [11]:
print(response.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ko">
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8">
    <link rel="canonical" href="http://land.naver.com">
    <meta property="og:site_name" content="land" />
    <meta property="og:title" content="네이버 부동산" />
    <meta property="og:type" content="article" />
    <meta property="og:description" content="내 집 마련의 시작, 네이버 부동산" />
    <meta name="description" content="내 집 마련의 시작, 네이버 부동산">
    <meta property="og:image" content="https://ssl.pstatic.net/static.land/static/service/20191007/lnb/ico_land_large_og_tag.png"/>
    <meta property="og:article:author" content="네이버 부동산" />
    <meta property="og:url" content="http://land.naver.com/index.nhn" />
    <meta property="og:article:author:url" content="http://land.naver.com/" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>

    

    <style type="text/css">
        html, body, div, p {
 

일반적인 크롤링방법으로는 막힘. fake_useragent 사용.  
- 패키지 설치 : `pip install fake-useragent`

In [13]:
from fake_useragent import UserAgent

In [15]:
headers = {
    'user-agent': UserAgent().chrome,
    'referer': 'https://new.land.naver.com/complexes?ms=37.637433,126.832359,15&a=APT:ABYG:JGC&e=RETAIL'
} # 개발자 도구에서 header 정보에서 확인 가능

response = requests.get(url, headers = headers) 
response

<Response [200]>

In [28]:
response.text #JSON 타입 테이블로 크롤링됨.

'[{"markerId":"102375","markerType":"COMPLEX","latitude":37.62513,"longitude":126.82441,"complexName":"태웅정다움","realEstateTypeCode":"APT","realEstateTypeName":"아파트","completionYearMonth":"200506","totalDongCount":1,"totalHouseholdCount":66,"floorAreaRatio":294,"minDealUnitPrice":1108,"maxDealUnitPrice":1196,"minLeaseUnitPrice":0,"maxLeaseUnitPrice":0,"minLeaseRate":0,"maxLeaseRate":0,"minArea":"94.87","maxArea":"94.87","minDealPrice":31500,"maxDealPrice":34000,"minLeasePrice":0,"maxLeasePrice":0,"minRentPrice":0,"maxRentPrice":0,"minShortTermRentPrice":0,"maxShortTermRentPrice":0,"isDealShown":true,"priceCount":1,"representativeArea":94.0,"medianDealUnitPrice":1108,"medianDealPrice":31500,"photoCount":0,"dealCount":3,"leaseCount":0,"rentCount":0,"shortTermRentCount":0,"totalArticleCount":3,"existPriceTab":true},{"markerId":"1570","markerType":"COMPLEX","latitude":37.62919,"longitude":126.834984,"complexName":"옥빛15단지풍산","realEstateTypeCode":"APT","realEstateTypeName":"아파트","completionYea

In [23]:
data = response.json()

In [29]:
result_df = pd.DataFrame(data) # Pandas로 데이터프레임화
result_df.tail()
# result_df.to_csv('real_estate.csv') # CSV 파일로 저장